In [1]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import tqdm
from scipy.stats import levene

# import anova and tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
def anova_function(features_df: pd.DataFrame, Metdata_column: str) -> pd.DataFrame:
    """
    This function will take in a dataframe and a metadata column and return the results of an anova and tukeyhsd test for each feature.


    Parameters
    ----------
    features_df : pd.DataFrame
        The dataframe containing the features with only one metadata column
    Metdata_column : str
        The name of the metadata column to be used for the anova test

    Returns
    -------
    pd.DataFrame
        A dataframe containing the results of the anova and tukeyhsd test for each feature
    """

    # anova and tukeyhsd for each feature
    # create a pandas data frame to store the results
    anova_results = pd.DataFrame()

    # loop through each feature
    for feature in tqdm.tqdm(features_df.columns[:-1]):
        # create a model
        model = ols(f"{feature} ~ C({Metdata_column})", data=features_df).fit()
        # create an anova table
        anova_table = sm.stats.anova_lm(model, typ=2)
        # create a tukeyhsd table
        tukeyhsd = pairwise_tukeyhsd(features_df[feature], features_df[Metdata_column])
        # get the f-statistic based p-value
        anova_p_value = anova_table["PR(>F)"][0]
        tmp = pd.DataFrame(
            tukeyhsd._results_table.data, columns=tukeyhsd._results_table.data[0]
        ).drop(0)
        tmp.reset_index(inplace=True, drop=True)
        # drop the first row
        tmp["feature"] = feature
        tmp["anova_p_value"] = anova_p_value
        tmp = pd.DataFrame(tmp)

        anova_results = pd.concat([anova_results, tmp], axis=0).reset_index(drop=True)
    return anova_results

In [3]:
file_path = pathlib.Path("../../data/5.converted_data/mean_aggregated_data.parquet")
df = pd.read_parquet(file_path)
df.head()

,Metadata_genotype,Metadata_replicate,Metadata_side,AreaShape_Area,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,...,Texture_SumEntropy_OP_3_02_256,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256
0,high,1,L,-0.178797,-0.178797,0.003558,-0.386796,0.280118,0.037998,0.434046,...,-0.627762,-0.644990,-0.903354,-0.890939,-0.896995,-0.902870,-0.896145,-0.898448,-0.898880,-0.896609
1,high,1,R,1.371763,1.371763,0.538195,0.288322,0.501618,4.163833,-1.361920,...,0.419551,0.406918,-0.188358,-0.159114,-0.189615,-0.186308,-0.218981,-0.225754,-0.219073,-0.225134
2,high,10,L,1.706234,1.706234,2.652373,3.280425,-1.992966,0.184630,-1.404376,...,0.841475,0.834574,1.149186,1.152200,1.078905,1.144848,1.040009,1.025740,1.045492,1.027617
3,high,10,R,0.771674,0.771674,-1.332747,0.164304,0.244371,0.173760,-1.885144,...,0.452480,0.436138,0.112178,0.131982,0.108602,0.125700,0.075717,0.071207,0.076477,0.067922
4,high,11,L,2.180858,2.180858,2.142686,1.838038,-0.077515,0.008650,0.876597,...,0.494813,0.490844,-0.016140,0.001041,-0.021997,-0.020992,-0.033934,-0.042635,-0.034983,-0.038763


In [4]:
# split the features and the metadata
metadata = df.columns.str.contains("Metadata")
# filter the metadata
metadata_df = df.loc[:, metadata]
# filter the features
features_df = df.loc[:, ~metadata]

## Anova for genotype only

In [5]:
anova_input_df = features_df.copy()
anova_input_df["Metadata_genotype"] = metadata_df["Metadata_genotype"]
anova_output_df = anova_function(anova_input_df, "Metadata_genotype")
print(anova_output_df.shape)
anova_output_df.head()

  0%|                                                                                                                                                     | 0/244 [00:00<?, ?it/s]

  0%|▌                                                                                                                                            | 1/244 [00:00<00:28,  8.65it/s]

  1%|█▏                                                                                                                                           | 2/244 [00:00<00:27,  8.78it/s]

  1%|█▋                                                                                                                                           | 3/244 [00:00<00:27,  8.93it/s]

  2%|██▎                                                                                                                                          | 4/244 [00:00<00:26,  8.94it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 243/244 [00:26<00:00,  8.67it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:26<00:00,  8.64it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:26<00:00,  9.23it/s]

(732, 9)


,group1,group2,meandiff,p-adj,lower,upper,reject,feature,anova_p_value
0,high,unsel,-1.5096,0.0,-1.8942,-1.125,True,AreaShape_Area,1.089545e-20
1,high,wt,-2.0158,0.0,-2.3969,-1.6347,True,AreaShape_Area,1.089545e-20
2,unsel,wt,-0.5062,0.0066,-0.8908,-0.1216,True,AreaShape_Area,1.089545e-20
3,high,unsel,-1.5096,0.0,-1.8942,-1.125,True,AreaShape_CentralMoment_0_0,1.089545e-20
4,high,wt,-2.0158,0.0,-2.3969,-1.6347,True,AreaShape_CentralMoment_0_0,1.089545e-20


In [6]:
# save the results
output_file = pathlib.Path(
    "../../data/6.analysis_results/mean_aggregated_anova_results.parquet"
)
output_file.parent.mkdir(exist_ok=True, parents=True)
anova_output_df.to_parquet(output_file)

## Levene's test for homogeneity of variance

In [7]:
# split the df into three genotypes
high_df = df[df["Metadata_genotype"] == "high"]
unsel_df = df[df["Metadata_genotype"] == "unsel"]
wt_df = df[df["Metadata_genotype"] == "wt"]
levene_test_results = {"feature": [], "levene_statistic": [], "levene_p_value": []}
for feature in tqdm.tqdm(features_df.columns):
    # calculate the levene test for each feature
    levene_results = levene(wt_df[feature], unsel_df[feature], high_df[feature])
    levene_test_results["feature"].append(feature)
    levene_test_results["levene_statistic"].append(levene_results.statistic)
    levene_test_results["levene_p_value"].append(levene_results.pvalue)

levene_test_results_df = pd.DataFrame(levene_test_results)
levene_test_results_df

  0%|                                                                          | 0/244 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████| 244/244 [00:00<00:00, 3781.56it/s]

,feature,levene_statistic,levene_p_value
0,AreaShape_Area,4.808285,0.010668
1,AreaShape_CentralMoment_0_0,4.808285,0.010668
2,AreaShape_CentralMoment_0_1,9.124611,0.000269
3,AreaShape_CentralMoment_0_2,9.782865,0.000158
4,AreaShape_CentralMoment_0_3,3.424782,0.037402
...,...,...,...
239,Texture_SumVariance_OP_3_03_256,7.546945,0.000994
240,Texture_Variance_OP_3_00_256,8.691200,0.000384
241,Texture_Variance_OP_3_01_256,8.906770,0.000322
242,Texture_Variance_OP_3_02_256,8.563326,0.000427


## Calculate the levenes test statistic for the equality of variances

In [8]:
# split the df into three genotypes
high_df = df[df["Metadata_genotype"] == "high"]
unsel_df = df[df["Metadata_genotype"] == "unsel"]
wt_df = df[df["Metadata_genotype"] == "wt"]
group_dict = {
    "high_vs_unsel": [high_df, unsel_df],
    "high_vs_wt": [high_df, wt_df],
    "unsel_vs_wt": [wt_df, unsel_df],
    "all": [high_df, unsel_df, wt_df],
}


levene_test_results = {
    "feature": [],
    "levene_statistic": [],
    "levene_p_value": [],
    "group": [],
}
for group in tqdm.tqdm(group_dict.keys()):
    for feature in features_df.columns:
        # calculate the levene test for each feature
        if not group == "all":
            levene_results = levene(
                group_dict[group][0][feature], group_dict[group][1][feature]
            )
            levene_test_results["feature"].append(feature)
            levene_test_results["levene_statistic"].append(levene_results.statistic)
            levene_test_results["levene_p_value"].append(levene_results.pvalue)
            levene_test_results["group"].append(group)
        else:
            levene_results = levene(
                group_dict[group][0][feature],
                group_dict[group][1][feature],
                group_dict[group][2][feature],
            )
            levene_test_results["feature"].append(feature)
            levene_test_results["levene_statistic"].append(levene_results.statistic)
            levene_test_results["levene_p_value"].append(levene_results.pvalue)
            levene_test_results["group"].append(group)

levene_test_results_df = pd.DataFrame(levene_test_results)

# sort the levene test results levene_test_results_df
# change the levene p-value to a float
levene_test_results_df["levene_p_value"] = levene_test_results_df[
    "levene_p_value"
].astype(float)
levene_test_results_df = levene_test_results_df.sort_values(
    "levene_p_value", ascending=False
)
levene_test_results_df

  0%|                                                                            | 0/4 [00:00<?, ?it/s]

/home/lippincm/miniconda3/envs/op_cell_processing_env/lib/python3.8/site-packages/scipy/stats/_morestats.py:2710: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom


 75%|███████████████████████████████████████████████████                 | 3/4 [00:00<00:00, 22.00it/s]

100%|████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 21.52it/s]

,feature,levene_statistic,levene_p_value,group
437,Texture_AngularSecondMoment_OP_3_01_256,0.000013,9.971029e-01,high_vs_wt
675,RadialDistribution_ZernikePhase_OP_9_1,0.000477,9.826518e-01,unsel_vs_wt
539,AreaShape_NormalizedMoment_3_2,0.000572,9.810111e-01,unsel_vs_wt
87,AreaShape_Zernike_7_7,0.001244,9.719976e-01,high_vs_unsel
436,Texture_AngularSecondMoment_OP_3_00_256,0.001246,9.719719e-01,high_vs_wt
...,...,...,...,...
887,RadialDistribution_ZernikeMagnitude_OP_8_2,26.848462,1.198789e-09,all
266,AreaShape_HuMoment_2,54.658521,9.472909e-10,high_vs_wt
872,RadialDistribution_ZernikeMagnitude_OP_4_0,27.984815,6.108337e-10,all
929,Texture_Contrast_OP_3_01_256,30.070757,1.823590e-10,all


In [9]:
# save the levene test results
# out dir
out_dir = pathlib.Path("../../data/6.analysis_results/")
# create the dir if it does not exist
out_dir.mkdir(parents=True, exist_ok=True)
levene_test_results_path = pathlib.Path(
    out_dir / "mean_aggregated_levene_test_results.csv"
)
levene_test_results_df.to_csv(levene_test_results_path)

### Calculate the levene test statistic for the aggregated data across feature types and genotypes

In [10]:
data_path = pathlib.Path(
    "../../data/5.converted_data/mean_aggregated_data.parquet"
).resolve(strict=True)
# Read the data
data = pd.read_parquet(data_path)

# Drop all metadata except for the genotype data
features_df = data.drop(columns=data.filter(like="Metadata").columns)
features_df["Metadata_genotype"] = data["Metadata_genotype"]


# turn the features into a long format
features_long_df = features_df.melt(
    id_vars="Metadata_genotype", var_name="feature", value_name="value"
)
features_long_df.head()
# Separate the feature into different parts
features_long_df[
    ["feature_group", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
] = features_long_df["feature"].str.split("_", expand=True)

# Replace the Metadata_genotype with the actual genotype name
features_long_df["Metadata_genotype"] = features_long_df["Metadata_genotype"].replace(
    {"high": "High-Severity", "unsel": "Mid-Severity", "wt": "Wild Type"}
)
features_long_df.head()

,Metadata_genotype,feature,value,feature_group,measurement,bone,parameter1,parameter2,parameter3
0,High-Severity,AreaShape_Area,-0.178797,AreaShape,Area,None,None,None,None
1,High-Severity,AreaShape_Area,1.371763,AreaShape,Area,None,None,None,None
2,High-Severity,AreaShape_Area,1.706234,AreaShape,Area,None,None,None,None
3,High-Severity,AreaShape_Area,0.771674,AreaShape,Area,None,None,None,None
4,High-Severity,AreaShape_Area,2.180858,AreaShape,Area,None,None,None,None


In [11]:
# break each genotype and featuretype into a separate dataframe
high_df = features_long_df[features_long_df["Metadata_genotype"] == "High-Severity"]
unsel_df = features_long_df[features_long_df["Metadata_genotype"] == "Mid-Severity"]
wt_df = features_long_df[features_long_df["Metadata_genotype"] == "Wild Type"]

# each feature group
high_df_AreaShape = high_df[high_df["feature_group"] == "AreaShape"]
high_df_Intensity = high_df[high_df["feature_group"] == "Intensity"]
high_df_Neighbors = high_df[high_df["feature_group"] == "Neighbors"]
high_df_radial = high_df[high_df["feature_group"] == "RadialDistribution"]
high_df_Granularity = high_df[high_df["feature_group"] == "Granularity"]

unsel_df_AreaShape = unsel_df[unsel_df["feature_group"] == "AreaShape"]
unsel_df_Intensity = unsel_df[unsel_df["feature_group"] == "Intensity"]
unsel_df_Neighbors = unsel_df[unsel_df["feature_group"] == "Neighbors"]
unsel_df_radial = unsel_df[unsel_df["feature_group"] == "RadialDistribution"]
unsel_df_Granularity = unsel_df[unsel_df["feature_group"] == "Granularity"]

wt_df_AreaShape = wt_df[wt_df["feature_group"] == "AreaShape"]
wt_df_Intensity = wt_df[wt_df["feature_group"] == "Intensity"]
wt_df_Neighbors = wt_df[wt_df["feature_group"] == "Neighbors"]
wt_df_radial = wt_df[wt_df["feature_group"] == "RadialDistribution"]
wt_df_Granularity = wt_df[wt_df["feature_group"] == "Granularity"]

# levene test for each feature group
levene_test_results = {
    "feature_group": [],
    "levene_statistic": [],
    "levene_p_value": [],
    "group": [],
}

group_dict = {
    "AreaShape": {
        "high_area_v_unsel_area": [high_df_AreaShape, unsel_df_AreaShape],
        "high_area_v_wt_area": [high_df_AreaShape, wt_df_AreaShape],
        "unsel_area_v_wt_area": [wt_df_AreaShape, unsel_df_AreaShape],
    },
    "Intensity": {
        "high_intensity_v_unsel_intensity": [high_df_Intensity, unsel_df_Intensity],
        "high_intensity_v_wt_intensity": [high_df_Intensity, wt_df_Intensity],
        "unsel_intensity_v_wt_intensity": [wt_df_Intensity, unsel_df_Intensity],
    },
    "Neighbors": {
        "high_neighbors_v_unsel_neighbors": [high_df_Neighbors, unsel_df_Neighbors],
        "high_neighbors_v_wt_neighbors": [high_df_Neighbors, wt_df_Neighbors],
        "unsel_neighbors_v_wt_neighbors": [wt_df_Neighbors, unsel_df_Neighbors],
    },
    "RadialDistribution": {
        "high_radial_v_unsel_radial": [high_df_radial, unsel_df_radial],
        "high_radial_v_wt_radial": [high_df_radial, wt_df_radial],
        "unsel_radial_v_wt_radial": [wt_df_radial, unsel_df_radial],
    },
    "Granularity": {
        "high_granularity_v_unsel_granularity": [
            high_df_Granularity,
            unsel_df_Granularity,
        ],
        "high_granularity_v_wt_granularity": [high_df_Granularity, wt_df_Granularity],
        "unsel_granularity_v_wt_granularity": [wt_df_Granularity, unsel_df_Granularity],
    },
}

for feature_group in tqdm.tqdm(group_dict.keys()):
    for group in group_dict[feature_group].keys():
        if not group == "all":
            levene_results = levene(
                group_dict[feature_group][group][0]["value"],
                group_dict[feature_group][group][1]["value"],
            )
            # calculate the variance for each feature group

            levene_test_results["feature_group"].append(feature_group)
            levene_test_results["levene_statistic"].append(levene_results.statistic)
            levene_test_results["levene_p_value"].append(levene_results.pvalue)
            levene_test_results["group"].append(group)
        else:
            pass

levene_test_results_df = pd.DataFrame(levene_test_results)
levene_test_results_df.head()

  0%|                                                                            | 0/5 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 864.16it/s]

,feature_group,levene_statistic,levene_p_value,group
0,AreaShape,392.323288,2.433017e-84,high_area_v_unsel_area
1,AreaShape,796.248155,1.069709e-163,high_area_v_wt_area
2,AreaShape,36.568230,1.573327e-09,unsel_area_v_wt_area
3,Intensity,65.355599,2.227985e-15,high_intensity_v_unsel_intensity
4,Intensity,0.834856,3.611348e-01,high_intensity_v_wt_intensity


In [12]:
# save the levene test results
# out dir
out_dir = pathlib.Path("../../data/6.analysis_results/")
# create the dir if it does not exist
out_dir.mkdir(parents=True, exist_ok=True)
levene_test_results_path = pathlib.Path(
    out_dir / "mean_aggregated_levene_test_results_feature_types.csv"
)
levene_test_results_df.to_csv(levene_test_results_path, index=False)

In [13]:
# Drop all metadata except for the genotype data
features_df = data.drop(columns=data.filter(like="Metadata").columns)
features_df["Metadata_genotype"] = data["Metadata_genotype"]

# turn the features into a long format
features_long_df = features_df.melt(
    id_vars="Metadata_genotype", var_name="feature", value_name="value"
)
features_long_df

,Metadata_genotype,feature,value
0,high,AreaShape_Area,-0.178797
1,high,AreaShape_Area,1.371763
2,high,AreaShape_Area,1.706234
3,high,AreaShape_Area,0.771674
4,high,AreaShape_Area,2.180858
...,...,...,...
20247,wt,Texture_Variance_OP_3_03_256,1.434540
20248,wt,Texture_Variance_OP_3_03_256,1.956497
20249,wt,Texture_Variance_OP_3_03_256,1.223845
20250,wt,Texture_Variance_OP_3_03_256,-1.017697


In [14]:
# get the variance for each feature group
var_df = features_long_df.groupby(["Metadata_genotype", "feature"]).var().reset_index()
var_df.head()
# change the value column name to variance
var_df.rename(columns={"value": "variance"}, inplace=True)

In [15]:
var_df[
    ["feature_group", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
] = var_df["feature"].str.split("_", expand=True)

# Replace the Metadata_genotype with the actual genotype name
var_df["Metadata_genotype"] = var_df["Metadata_genotype"].replace(
    {"high": "High-Severity", "unsel": "Mid-Severity", "wt": "Wild Type"}
)
var_df
var_df = var_df.drop(
    columns=["feature", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
)
var_df
# save the variance results
var_path = pathlib.Path(out_dir / "mean_aggregated_variance_results_feature_types.csv")
var_df.to_csv(var_path, index=False)

In [16]:
# get the mean and stdev for each feature group's variance
var_df = (
    var_df.groupby(["Metadata_genotype", "feature_group"])
    .agg(["mean", "std", "max", "min", "count"])
    .reset_index()
)
# ungroup the columns
var_df.columns = ["_".join(col).strip() for col in var_df.columns.values]
# rename the Metadata_genotype_ column and the feature_group_ column
var_df.rename(
    columns={
        "Metadata_genotype_": "Metadata_genotype",
        "feature_group_": "feature_group",
    },
    inplace=True,
)
var_df

,Metadata_genotype,feature_group,variance_mean,variance_std,variance_max,variance_min,variance_count
0,High-Severity,AreaShape,0.774588,0.669970,3.855982,0.000005,98
1,High-Severity,Granularity,0.278190,0.357954,0.981199,0.015287,6
2,High-Severity,Intensity,0.529629,0.281565,1.157014,0.129043,15
3,High-Severity,Neighbors,0.730238,0.237898,0.953424,0.479949,3
4,High-Severity,RadialDistribution,0.670404,0.430295,1.751219,0.000000,70
5,High-Severity,Texture,0.300315,0.206077,0.719217,0.032302,52
6,Mid-Severity,AreaShape,0.608826,0.372412,2.166008,0.160115,98
7,Mid-Severity,Granularity,0.314790,0.143204,0.539786,0.191056,6
8,Mid-Severity,Intensity,0.285852,0.211016,0.824411,0.076053,15
9,Mid-Severity,Neighbors,1.025230,0.424006,1.469152,0.624438,3


In [17]:
# save the variance results
var_path = pathlib.Path(
    out_dir / "mean_aggregated_variance_results_feature_types_stats.csv"
)
var_df.to_csv(var_path, index=False)